---
# Automatically generate new Python files from the Spoonacular API info page
---

In [11]:
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Start the Selenium browser (Chrome)

In [12]:
# Start a new instance of Chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
browser = webdriver.Chrome('/usr/local/bin/chromedriver', chrome_options=chrome_options)
url = "https://spoonacular.com/food-api"
browser.get(url)
time.sleep(1)

# Scroll down to list of endpoints (necessary?)
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 8
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    no_of_pagedowns-=1
    
print("Waiting for endpoints to load...")
time.sleep(5)

Waiting for endpoints to load...


## Scrape documentation for each endpoint

In [13]:
def getEndpointInfo(save=True):
    endpoints = []
    m,p = 'mashape-', 'parameter-'
    regex = re.compile(r'required|optional', re.VERBOSE)    
    doc = browser.find_element_by_class_name(m+'doc')
    list_of_endpoints = doc.find_elements_by_class_name(m+'endpoint')
    del(list_of_endpoints[36])
    num_eps = len(list_of_endpoints)
    print("Found {num} endpoints to scrape.".format(num=num_eps))
    
    for n,ep in enumerate(list_of_endpoints):
        ep.click()
        time.sleep(0.5)
        endpoint = {}
        
        # Get info on the endpoint name and route
        endpoint['name'] = ep.find_element_by_class_name(m+'endpoint-name').text
        print('-'*30 + '\n({n}/{t}) "{name}"'.format(name=endpoint['name'], n=n+1, t=num_eps))
        content = ep.find_element_by_class_name(m+'endpoint-content')
        route = content.find_element_by_class_name(m+'endpoint-route').text.split(' /')
        endpoint['method'] = route[0]
        endpoint['uri'] = route[1]
        time.sleep(2)
        
        # Get info on the endpoint parameters
        parameters = []
        print("Params:")
        params = content.find_elements_by_class_name(m+'parameter')
        for param in params:
            parameter = {}    
            if param.text != '':
                for s in param.find_elements_by_tag_name('span'):
                    var = s.get_attribute('class').split(m+p,1)[-1].lower()
                    if 'condition' in var:
                        var = 'condition'
                        val = regex.search(s.text).group()
                    elif 'example' in var:
                        val = s.text.split('Example:',1)[-1]
                    else:
                        val = s.text                    
                    parameter[var] = val       

                try:    
                    parameter['response'] = content.find_element_by_class_name(m+'example').text
                except:
                    parameter['response'] = None
                print("\t{name}: {type}".format(name=parameter['name'], type=parameter['type']))
            else:
                print("\t\t***MISSING PARAMETER***")
                parameter = {'MISSING': True}
            parameters.append(parameter)
            
        # Add info on current endpoint to list of all endpoints
        endpoint['parameters'] = parameters
        endpoints.append(endpoint)
                
        if save and ((n)%10==0 or n+1==len(list_of_endpoints)):
            filename = 'spoonacular_api_names_and_endpoints'
            with open(filename + '.json', 'w') as outfile:
                json.dump(endpoints, outfile)                

    return endpoints                   

In [14]:
all_info = getEndpointInfo()
all_info[-1]

Found 52 endpoints to scrape.
------------------------------
(1/52) "Analyze a Recipe Search Query"
Params:
	q: STRING
------------------------------
(2/52) "Analyze Recipe Instructions"
Params:
	instructions: STRING
------------------------------
(3/52) "Detect Food in Text"
Params:
	text: STRING
------------------------------
(4/52) "Extract Recipe from Website"
Params:
	forceExtraction: BOOLEAN
	url: STRING
------------------------------
(5/52) "Parse Ingredients"
Params:
	includeNutrition: BOOLEAN
	ingredientList: STRING
	servings: NUMBER
------------------------------
(6/52) "Autocomplete Ingredient Search"
Params:
	intolerances: STRING
	metaInformation: BOOLEAN
	number: NUMBER
	query: STRING
------------------------------
(7/52) "Autocomplete Recipe Search"
Params:
	number: NUMBER
	query: STRING
------------------------------
(8/52) "Get Comparable Products"
Params:
	upc: STRING
------------------------------
(9/52) "Get Dish Pairing for Wine"
Params:
	wine: STRING
--------------

------------------------------
(31/52) "Generate Meal Plan"
Params:
	diet: STRING
	exclude: STRING
	targetCalories: NUMBER
	timeFrame: STRING
------------------------------
(32/52) "Guess Nutrition by Dish Name"
Params:
	title: STRING
------------------------------
(33/52) "Map Ingredients to Grocery Products"
Params:
------------------------------
(34/52) "Match Recipes to Daily Calories"
Params:
	targetCalories: NUMBER
	timeFrame: STRING
------------------------------
(35/52) "Quick Answer"
Params:
	q: STRING
------------------------------
(36/52) "Summarize Recipe"
Params:
	id: NUMBER
------------------------------
(37/52) "Visualize Ingredients"
Params:
	defaultCss: BOOLEAN
	ingredientList: STRING
	measure: STRING
	servings: NUMBER
	showBacklink: BOOLEAN
	view: STRING
------------------------------
(38/52) "Visualize Menu Item Nutrition"
Params:
	defaultCss: BOOLEAN
	id: NUMBER
------------------------------
(39/52) "Visualize Price Breakdown"
Params:
	defaultCss: BOOLEAN
	ingredie

{'name': 'Talk to a chatbot',
 'method': 'GET',
 'uri': 'food/converse',
 'parameters': [{'name': 'contextId',
   'type': 'STRING',
   'condition': 'optional',
   'description': 'An arbitrary globally unique id for your conversation. The conversation can contain states so you should pass your context id if you want the bot to be able to remember the conversation.',
   'example': '342938',
   'response': '{"answerText":"Here are some donut recipes for you.","media":[{"title":"Homemade Chinese Doughnuts","image":"https://spoonacular.com/recipeImages/797495-312x231.jpg","link":"https://spoonacular.com/recipes/homemade-chinese-doughnuts-797495"},{"title":"Baked Lemon Blueberry Doughnuts (Donuts)","image":"https://spoonacular.com/recipeImages/775815-312x231.jpg","link":"https://spoonacular.com/recipes/baked-lemon-blueberry-doughnuts-donuts-775815"},{"title":"Boston Cream Donuts","image":"https://spoonacular.com/recipeImages/558271-312x231.jpg","link":"https://spoonacular.com/recipes/boston-

# Generate the Python wrapper from scraped API documentation